# Most Polluted Countries - Web Scraping 

### · Antes de empezar... 

1. Para gestionar los Dataframes:

In [1]:
pip install plotly.express

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np

/Users/paulaelizagarate/opt/anaconda3/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


2. Para hacer web scraping con Selenium 

In [4]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [4]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from time import sleep

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
from tqdm import tqdm

De cara a trabajar con Selenium

In [9]:
opciones= Options()
opciones.add_experimental_option('excludeSwitches', ['enable-automation'])

opciones.add_experimental_option('useAutomationExtension', False)
opciones.add_argument('--start-maximized') #empezar maximizado
opciones.add_argument('user.data-dir=selenium') #guarda las cookies
opciones.add_argument('--incognito')#incognito window

In [10]:
driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - Downloading:  99%|█████████▊| 8.60M/8.72M [00:03<00:00, 2.90MB/s]

###  · Carga de CSVs

Empezamos cargando datos CSV procedentes de Kaggle con la información de los países y las ciudades más contaminadas desde 2017 hasta 2021. 

In [11]:
df_pcountries = pd.read_csv("datos/AIR QUALITY INDEX- top countries.csv", index_col = 0 )
df_pcountries.head(2)

,Country/Region,2021,2020,2019,2018,Population
Rank,,,,,,
1,Bangladesh,76.9,77.1,83.3,97.1,"164,689,383"
2,Chad,75.9,-,-,-,"16,425,859"


In [12]:
df_pcities = pd.read_csv("datos/AIR QUALITY INDEX (by cities) - IQAir.csv", index_col = 0 )
df_pcities.head(2)

,City,2021,JAN(2021),FEB(2021),MAR(2021),APR(2021),MAY(2021),JUN(2021),JUL(2021),AUG(2021),SEP(2021),OCT(2021),NOV(2021),DEC(2021),2020,2019,2018,2017
Rank,,,,,,,,,,,,,,,,,,
1,"Bhiwadi, India",106.2,145.8,129.8,120.2,125.7,86.5,95.9,55.6,55.4,37.1,91.1,188.6,136.6,95.5,83.4,125.4,-
2,"Ghaziabad, India",102.0,199.9,172.2,97.8,86.3,52.9,47.2,35.3,37.6,30.8,89.7,218.3,163,106.6,110.2,135.2,144.6


[WDM] - Downloading: 100%|██████████| 8.72M/8.72M [00:20<00:00, 2.90MB/s]

### · Web Scraping Selenium 

Nuestro objetivo es scrapear los datos de natalidad de distintos años/países del Expansión, para ello utilizaremos Selenium. Iniciamos estructurando como haríamos la extracción de uno de los años. En este caso el más inmediato, 2021:

In [48]:
# iniciamos el driver
driver = webdriver.Chrome(ChromeDriverManager().install())

## accedemos a la pagina web
driver.get("https://datosmacro.expansion.com/demografia/natalidad?anio=2021")
driver.implicitly_wait(10)

### aceptamos las cookies
driver.find_element("css selector", "#didomi-notice-agree-button").click()

driver.implicitly_wait(5)

natalidad= driver.find_element("css selector", "#tb1").text


Una vez realizado el código para extraer los datos de natalidad de un año, podremos automatizar el proceso en el resto de años. Para ello, primero, tendremos que generar las diferentes URLs correspondientes a los diferentes años de los que queremos información y almacenarlas en una lista. Una vez tengamos la lista, iteraremos por ella, entrando así en cada enlace y aplicando el código genérico creado. Para evitar que tengamos un error que nos detenga el proceso, tendremos en cuenta que no en todas las URLs nos solicitarán que aceptemos cookies y que, de manera excepcional, saltá un pop up que tendremos que cerrar:

In [14]:
url_list = []

for year in range(2017, 2022):
        url_list.append(f"https://datosmacro.expansion.com/demografia/natalidad?anio={year}")

In [15]:
url_list

['https://datosmacro.expansion.com/demografia/natalidad?anio=2017',
 'https://datosmacro.expansion.com/demografia/natalidad?anio=2018',
 'https://datosmacro.expansion.com/demografia/natalidad?anio=2019',
 'https://datosmacro.expansion.com/demografia/natalidad?anio=2020',
 'https://datosmacro.expansion.com/demografia/natalidad?anio=2021']

In [16]:
driver = webdriver.Chrome(ChromeDriverManager().install())
result_list = []

for i in tqdm(url_list[:5]): 
    driver.get(i)
    try:
        sleep(5)
        driver.find_element("css selector", "#didomi-notice-agree-button").click()
        sleep(5)
        natalidad= driver.find_element("css selector", "#tb1").text
        result_list.append(natalidad)
    except:
        try:
            sleep(5)
            driver.find_element("xpath", '//*[@id="socModal"]/div[2]/div/div[1]/button/span').click()
            sleep(5)
            natalidad= driver.find_element("css selector", "#tb1").text
            result_list.append(natalidad)
        except:
            sleep(5)
            natalidad= driver.find_element("css selector", "#tb1").text
            result_list.append(natalidad)
driver.quit()


100%|██████████| 5/5 [04:08<00:00, 49.68s/it]


In [34]:
#result_list

### · Creación de Dataframe

Una vez obtenidos los datos, nuestro siguiente objetivo será la transformación de la lista a dataframe. Haremos una división de la información por año y almacenada en distintos DataFrames que más tarde uniremos. De este último paso tendremos como resultado los siguientes CSVs: `natalidad2017`, `natalidad2018`, `natalidad2019`, `natalidad2020` y `natalidad2021`.

In [18]:
df = pd.DataFrame(result_list)
df

,0
0,Países Fecha Nacidos Nacidos Hombres Nacidos M...
1,Países Fecha Nacidos Nacidos Hombres Nacidos M...
2,Países Fecha Nacidos Nacidos Hombres Nacidos M...
3,Países Fecha Nacidos Nacidos Hombres Nacidos M...
4,Países Fecha Nacidos Nacidos Hombres Nacidos M...


In [19]:
df.shape

(5, 1)

#### 2017

In [50]:
a = df.iloc[0][0].split("\n")
a[:5]

['Países Fecha Nacidos Nacidos Hombres Nacidos Mujeres Tasa Natalidad Índice de Fecund. Var.',
 'España [+] 2017 393.181 202.478 190.703 8,41‰ 1,31 -1,92%',
 'Alemania [+] 2017 784.901 402.517 382.384 9,50‰ 1,57 -1,88%',
 'Reino Unido [+] 2017 754.754 387.030 367.724 11,40‰ 1,74 -2,79%',
 'Francia [+] 2017 770.045 394.058 375.987 11,50‰ 1,89 -1,56%']

In [21]:
nat2017= pd.DataFrame(a)
nat2017

,0
0,Países Fecha Nacidos Nacidos Hombres Nacidos M...
1,"España [+] 2017 393.181 202.478 190.703 8,41‰ ..."
2,"Alemania [+] 2017 784.901 402.517 382.384 9,50..."
3,Reino Unido [+] 2017 754.754 387.030 367.724 1...
4,"Francia [+] 2017 770.045 394.058 375.987 11,50..."
...,...
189,"Samoa [+] 2017 24,69‰ 3,93 -1,26%"
190,"Yemen [+] 2017 31,04‰ 3,89 -2,60%"
191,"Sudáfrica [+] 2017 20,91‰ 2,43 -1,10%"
192,"Zambia [+] 2017 36,70‰ 4,72 -1,99%"


In [22]:
dfn17=nat2017[1:]
dfn17

,0
1,"España [+] 2017 393.181 202.478 190.703 8,41‰ ..."
2,"Alemania [+] 2017 784.901 402.517 382.384 9,50..."
3,Reino Unido [+] 2017 754.754 387.030 367.724 1...
4,"Francia [+] 2017 770.045 394.058 375.987 11,50..."
5,"Italia [+] 2017 458.151 235.733 222.418 7,60‰ ..."
...,...
189,"Samoa [+] 2017 24,69‰ 3,93 -1,26%"
190,"Yemen [+] 2017 31,04‰ 3,89 -2,60%"
191,"Sudáfrica [+] 2017 20,91‰ 2,43 -1,10%"
192,"Zambia [+] 2017 36,70‰ 4,72 -1,99%"


In [54]:
dfn17.to_csv('natalidad2017.csv')

In [51]:
n17= dfn17[0].str.split("", expand=True)
n17

,0,1
1,España [+,"2017 393.181 202.478 190.703 8,41‰ 1,31 -1,92%"
2,Alemania [+,"2017 784.901 402.517 382.384 9,50‰ 1,57 -1,88%"
3,Reino Unido [+,"2017 754.754 387.030 367.724 11,40‰ 1,74 -2,79%"
4,Francia [+,"2017 770.045 394.058 375.987 11,50‰ 1,89 -1,56%"
5,Italia [+,"2017 458.151 235.733 222.418 7,60‰ 1,32 -1,49%"
...,...,...
189,Samoa [+,"2017 24,69‰ 3,93 -1,26%"
190,Yemen [+,"2017 31,04‰ 3,89 -2,60%"
191,Sudáfrica [+,"2017 20,91‰ 2,43 -1,10%"
192,Zambia [+,"2017 36,70‰ 4,72 -1,99%"


In [24]:
#columnas= [nat2017[0][0]]
#nat2017.columns= columnas

In [25]:
#columnas=[i.lower() for i in columnas]
#columnas

['países fecha nacidos nacidos hombres nacidos mujeres tasa natalidad índice de fecund. var.']

#### 2018 

In [26]:
b = df.iloc[1][0].split("\n")
b[:5]

['Países Fecha Nacidos Nacidos Hombres Nacidos Mujeres Tasa Natalidad Índice de Fecund. Var.',
 'España [+] 2018 372.777 191.569 181.208 7,94‰ 1,26 -3,67%',
 'Alemania [+] 2018 787.523 404.052 383.471 9,50‰ 1,57 0',
 'Reino Unido [+] 2018 730.918 375.379 355.539 11,00‰ 1,68 -3,45%',
 'Francia [+] 2018 759.199 388.944 370.255 11,30‰ 1,87 -1,06%']

In [27]:
nat2018= pd.DataFrame(b)
nat2018

,0
0,Países Fecha Nacidos Nacidos Hombres Nacidos M...
1,"España [+] 2018 372.777 191.569 181.208 7,94‰ ..."
2,"Alemania [+] 2018 787.523 404.052 383.471 9,50..."
3,Reino Unido [+] 2018 730.918 375.379 355.539 1...
4,"Francia [+] 2018 759.199 388.944 370.255 11,30..."
...,...
189,"Samoa [+] 2018 24,38‰ 3,88 -1,25%"
190,"Yemen [+] 2018 30,45‰ 3,79 -2,52%"
191,"Sudáfrica [+] 2018 20,51‰ 2,41 -1,03%"
192,"Zambia [+] 2018 36,19‰ 4,63 -1,80%"


In [28]:
dfn18= nat2018[1:]
dfn18

,0
1,"España [+] 2018 372.777 191.569 181.208 7,94‰ ..."
2,"Alemania [+] 2018 787.523 404.052 383.471 9,50..."
3,Reino Unido [+] 2018 730.918 375.379 355.539 1...
4,"Francia [+] 2018 759.199 388.944 370.255 11,30..."
5,"Italia [+] 2018 439.747 226.217 213.530 7,30‰ ..."
...,...
189,"Samoa [+] 2018 24,38‰ 3,88 -1,25%"
190,"Yemen [+] 2018 30,45‰ 3,79 -2,52%"
191,"Sudáfrica [+] 2018 20,51‰ 2,41 -1,03%"
192,"Zambia [+] 2018 36,19‰ 4,63 -1,80%"


In [55]:
dfn18.to_csv('natalidad2018.csv')

#### 2019

In [30]:
c = df.iloc[2][0].split("\n")
c[:5]

['Países Fecha Nacidos Nacidos Hombres Nacidos Mujeres Tasa Natalidad Índice de Fecund. Var.',
 'España [+] 2019 360.617 185.523 175.094 7,62‰ 1,24 -2,24%',
 'Alemania [+] 2019 778.090 399.292 378.798 9,40‰ 1,54 -1,91%',
 'Reino Unido [+] 2019 712.699 375.379 355.539 10,70‰ 1,63 -2,98%',
 'Francia [+] 2019 754.008 385.340 368.668 11,20‰ 1,86 -0,53%']

In [31]:
nat2019= pd.DataFrame(c)
nat2019

,0
0,Países Fecha Nacidos Nacidos Hombres Nacidos M...
1,"España [+] 2019 360.617 185.523 175.094 7,62‰ ..."
2,"Alemania [+] 2019 778.090 399.292 378.798 9,40..."
3,Reino Unido [+] 2019 712.699 375.379 355.539 1...
4,"Francia [+] 2019 754.008 385.340 368.668 11,20..."
...,...
189,"Samoa [+] 2019 24,10‰ 3,83 -1,21%"
190,"Yemen [+] 2019 29,87‰ 3,70 -2,43%"
191,"Sudáfrica [+] 2019 20,13‰ 2,38 -1,00%"
192,"Zambia [+] 2019 35,78‰ 4,56 -1,60%"


In [32]:
dfn19= nat2019[1:]
dfn19

,0
1,"España [+] 2019 360.617 185.523 175.094 7,62‰ ..."
2,"Alemania [+] 2019 778.090 399.292 378.798 9,40..."
3,Reino Unido [+] 2019 712.699 375.379 355.539 1...
4,"Francia [+] 2019 754.008 385.340 368.668 11,20..."
5,"Italia [+] 2019 420.084 215.387 204.697 7,00‰ ..."
...,...
189,"Samoa [+] 2019 24,10‰ 3,83 -1,21%"
190,"Yemen [+] 2019 29,87‰ 3,70 -2,43%"
191,"Sudáfrica [+] 2019 20,13‰ 2,38 -1,00%"
192,"Zambia [+] 2019 35,78‰ 4,56 -1,60%"


In [56]:
dfn19.to_csv('natalidad2019.csv')

#### 2020

In [34]:
d = df.iloc[3][0].split("\n")
d[:5]

['Países Fecha Nacidos Nacidos Hombres Nacidos Mujeres Tasa Natalidad Índice de Fecund. Var.',
 'España [+] 2020 341.315 174.842 166.473 7,19‰ 1,19 -4,03%',
 'Alemania [+] 2020 773.144 397.385 375.759 9,30‰ 1,53 -0,65%',
 'Reino Unido [+] 2020 712.699 375.379 355.539 10,20‰ 1,56 -4,29%',
 'Francia [+] 2020 735.775 376.187 359.588 10,90‰ 1,83 -1,61%']

In [35]:
nat2020= pd.DataFrame(d)
nat2020

,0
0,Países Fecha Nacidos Nacidos Hombres Nacidos M...
1,"España [+] 2020 341.315 174.842 166.473 7,19‰ ..."
2,"Alemania [+] 2020 773.144 397.385 375.759 9,30..."
3,Reino Unido [+] 2020 712.699 375.379 355.539 1...
4,"Francia [+] 2020 735.775 376.187 359.588 10,90..."
...,...
189,"Samoa [+] 2020 23,86‰ 3,79 -1,17%"
190,"Yemen [+] 2020 29,30‰ 3,61 -2,32%"
191,"Sudáfrica [+] 2020 19,77‰ 2,36 -0,97%"
192,"Zambia [+] 2020 35,44‰ 4,50 -1,38%"


In [36]:
dfn20= nat2020[1:]
dfn20

,0
1,"España [+] 2020 341.315 174.842 166.473 7,19‰ ..."
2,"Alemania [+] 2020 773.144 397.385 375.759 9,30..."
3,Reino Unido [+] 2020 712.699 375.379 355.539 1...
4,"Francia [+] 2020 735.775 376.187 359.588 10,90..."
5,"Italia [+] 2020 404.892 208.431 196.461 6,80‰ ..."
...,...
189,"Samoa [+] 2020 23,86‰ 3,79 -1,17%"
190,"Yemen [+] 2020 29,30‰ 3,61 -2,32%"
191,"Sudáfrica [+] 2020 19,77‰ 2,36 -0,97%"
192,"Zambia [+] 2020 35,44‰ 4,50 -1,38%"


In [57]:
dfn20.to_csv('natalidad2020.csv')

#### 2021

In [38]:
f = df.iloc[4][0].split("\n")
f[:5]

['Países Fecha Nacidos Nacidos Hombres Nacidos Mujeres Tasa Natalidad Índice de Fecund. Var.',
 'España [+] 2021 337.380 174.148 163.232 7,12‰ 1,19 0,47%',
 'Alemania [+] 2021 795.492 397.385 375.759 9,60‰ 1,53 -0,65%',
 'Reino Unido [+] 2020 712.699 375.379 355.539 10,20‰ 1,56 -4,29%',
 'Francia [+] 2021 738.594 376.187 359.588 10,90‰ 1,83 -1,61%']

In [39]:
nat2021= pd.DataFrame(f)
nat2021

,0
0,Países Fecha Nacidos Nacidos Hombres Nacidos M...
1,"España [+] 2021 337.380 174.148 163.232 7,12‰ ..."
2,"Alemania [+] 2021 795.492 397.385 375.759 9,60..."
3,Reino Unido [+] 2020 712.699 375.379 355.539 1...
4,"Francia [+] 2021 738.594 376.187 359.588 10,90..."
...,...
189,"Samoa [+] 2020 23,86‰ 3,79 -1,17%"
190,"Yemen [+] 2020 29,30‰ 3,61 -2,32%"
191,"Sudáfrica [+] 2020 19,77‰ 2,36 -0,97%"
192,"Zambia [+] 2020 35,44‰ 4,50 -1,38%"


In [40]:
dfn21= nat2021[1:]
dfn21

,0
1,"España [+] 2021 337.380 174.148 163.232 7,12‰ ..."
2,"Alemania [+] 2021 795.492 397.385 375.759 9,60..."
3,Reino Unido [+] 2020 712.699 375.379 355.539 1...
4,"Francia [+] 2021 738.594 376.187 359.588 10,90..."
5,"Italia [+] 2021 399.431 208.431 196.461 6,80‰ ..."
...,...
189,"Samoa [+] 2020 23,86‰ 3,79 -1,17%"
190,"Yemen [+] 2020 29,30‰ 3,61 -2,32%"
191,"Sudáfrica [+] 2020 19,77‰ 2,36 -0,97%"
192,"Zambia [+] 2020 35,44‰ 4,50 -1,38%"


In [58]:
dfn21.to_csv('natalidad2021.csv')